In [144]:
import numpy as np
import pandas as pd

In [145]:
data = pd.read_csv("./Data/soc-sign-bitcoinotc.csv")
data.columns = ['source','target','rating','time']

In [146]:
data_new = data
data_new = data_new.loc[data_new['rating'] >= 8]
new_list_concact = list(set(data_new.source).union(set(data_new.target)))
data_new.index = np.arange(0,len(data_new))
data_new.head()

,source,target,rating,time
0,13,16,8,1.289254e+09
1,13,10,8,1.289254e+09
2,21,1,8,1.289441e+09
3,21,10,8,1.289441e+09
4,21,8,9,1.289441e+09


CONSTANTS

In [147]:
D = 0.15
D_COMP = 0.85
interations = data_new.shape[0]

In [148]:
adj_dict = dict()
def dict_populate():
    for index in range(interations):
        source = new_list_concact.index(data_new.loc[index,'source'])
        target = new_list_concact.index(data_new.loc[index,'target'])
        if (source in adj_dict):
            adj_dict[source].append(target)
        else:
            adj_dict[source] = [target]
    
        if (target not in adj_dict):
            adj_dict[target] = []
        
dict_populate()

In [149]:
def do_matrix_transition(matriz):
    for i in adj_dict.keys():
        if(len(adj_dict[i]) > 0):
            weitgh = float(1)/float(len(adj_dict[i]))
            for j in adj_dict[i]:
                transition[j][i] = weitgh

Creating matrices

In [150]:
transition = np.zeros((len(new_list_concact), len(new_list_concact)))
faz_matriz(transition)
matrix_factor = (float(1)/float(len(new_list_concact)) * np.ones(shape=(len(new_list_concact),len(new_list_concact))))

matrix_factor_np = np.matrix(matrix_factor)

m = D_COMP * transition + D * matrix_factor_np

v = (float(1)/float(len(transition)) * np.ones(shape=(len(new_list_concact),1)))


PageRank Calc

In [186]:
count = 0 

def pagerank(v):
    global count
    if (sum(abs(m*v - v)) > 0.001) :
        count += 1
        return pagerank(m*v)
    else:
        count +=1
        return m*v

QUESTÃO 1:

Quantas iterações o PageRank precisou rodar até atingir convergência?

In [188]:
result = pagerank(v)
print(count)

35


QUESTÃO 2:

Quais os 5 investidores mais importantes segundo o PageRank? Quais seus valores de PageRank?

In [183]:
def generate_unidimensional_result():
    new_list = []
    for i in result:
        new_list.append(i.item())
    return new_list

new_result = generate_unidimensional_result()
result_data_frame = pd.DataFrame({'ID': new_list_concact, 'PageRank': new_result})
result_data_frame.sort_values('PageRank', ascending=False).take(range(5))

,ID,PageRank
0,1,0.000139
120,202,0.000125
94,144,0.000117
887,3996,0.000093
182,361,0.000092


QUESTÃO 3:

Como você poderia usar o PageRank caso você fosse um investidor em bitcoins?

GEPHI